# Dicom file manipulation with the Pydicom Library
https://pydicom.github.io/

## Contents:
* Reading Dicom files (general)
* Changing Dicom tags
* Modifying Dicom images
* Reading Dicom CT image files
* Reading Dicom RT Plan files

###  Architecture of DICOM :
https://dicom.innolitics.com/ciods

# Reading Dicom files (general)

## Exemple with a Nuclear Medicine imaging (SPECT) Dicom

In [ ]:
import pydicom as dcm

# open dicom file (works for all .dcm file)
File = dcm.read_file('data/patient_SPECT.dcm')
print(File)

## Acces to Dicom data by sequence name 

In [ ]:
print('Image type :',  File.ImageType )
print('Image type first element :', File.ImageType[0])
print('Modality: ',  File.Modality)
print()
# Type of data
# Note that they are all string, so you need a conversion to (float, int...)
print('type of ImageType dicom sequence : ', type(File.ImageType))
print('type of first element of ImageType dicom sequence : ', type(File.ImageType[0]))
print()
pixelSpacing = File.PixelSpacing
print('pixel spacing :', pixelSpacing)
print('length pixel spacing sequence :', len(pixelSpacing))
print('type pixel spacing :', type(pixelSpacing[0]))

In [ ]:
# Search dicom sequence with 'name' chain
print(File.dir('name'))

## Acces to Dicom data by tag (hexadecimal)

In [ ]:
# print 'Detector Information Sequence' dicom sequence 
print(File[0x054, 0x022])

# print 'Radialposition' First sub sequence of 'Detector Information Sequence' dicom sequence
print(File[0x054, 0x022][0][0x018, 0x1142].value)

In [ ]:
list_0 = File[0x054, 0x022][0][0x018, 0x1142].value
list_1 = File[0x054, 0x022][1][0x018, 0x1142].value
# Note the loop to convert string into float into the 'list' pos_radial_0
pos_radial_0 = [float(val) for val in list_0]
pos_radial_1 = [float(val) for val in list_1]

# concatenate 2 lists:
pos_radial = pos_radial_1 + pos_radial_0
print(pos_radial)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot the positions
plt.plot(pos_radial, 'o-', label='detecteur 1+2');

In [ ]:
import numpy as np

# this polar plot with the same values describes camera trajectory surronding patient
plt.polar([2*np.pi*l/64 for l in range(64)],pos_radial);

## Exercise - Creation of a dictionary on the  Energy Windows (there are 5)
* search for the energy keyword in the tags
* find the tag on energy windows and the information on the lower and higher limit
* buid a dictionnary with the pic energy name as key and low, high and width as values

In [ ]:
# complete here

## Acces to Image pixel map

In [ ]:
# Acces to image pixel data 
arr_image = File.pixel_array
print(type(arr_image))
print('Image Dimension: ', File.pixel_array.shape)
print(File.pixel_array.min(), File.pixel_array.max(), File.pixel_array.mean())

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline

def im_viewer(im,cmap):
    def plot_image(myslice):
        plt.imshow(im[myslice,:,:], cmap=cmap)
    interact(plot_image, myslice=(0, im.shape[0]-1))
    return;

im_viewer(arr_image, plt.cm.hot)

# Modifying Dicom files

## Changing Dicom tags

In [ ]:
# Remove patient name
print(File.PatientName)
File.PatientName = 'PythonCourse'
print(File.PatientName)

## Changing the image data

In [ ]:
# modify and save
print(File.pixel_array.min(), File.pixel_array.max(), File.pixel_array.mean())
File.pixel_array[File.pixel_array < 100] = 0
print(File.pixel_array.min(), File.pixel_array.max(), File.pixel_array.mean())

# the pixel values are contained in PixelData, pixel_array is an attribute from pydicom
# to store the new pixel values they need to be set as a string to PixelData
File.PixelData = File.pixel_array.tostring()

## Saving Dicom file

In [ ]:
File.save_as('output/patient_SPECT_modified.dcm')

# Reading Dicom CT image

In [ ]:
import numpy as np
import pydicom as dcm
from glob import glob

def read_CT(path):
    # read all .dcm images and put them in a list slices
    slices = [dcm.read_file(file) for file in glob(path + '/*.dcm')]
    # sort the slices
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    # create a numpy matrix containing all slices
    ct = np.stack([s.pixel_array for s in slices], axis=-1)
    # Convert to Hounsfield units (HU)
    ct = ct * slices[0].RescaleSlope + slices[0].RescaleIntercept
    return ct

In [ ]:
# subset of anonymized CT dicom sample from https://www.dicomlibrary.com
ct_image = read_CT('data/patient_CT_dcm/')
ct_image.shape

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline

def im_viewer(im,cmap):
    def plot_image(myslice):
        plt.imshow(im[:,:,myslice], cmap=cmap)
    interact(plot_image, myslice=(0, im.shape[2]-1))
    return;

im_viewer(ct_image, plt.cm.bone)

# Reading DICOM RT plan

#### DICOM input for GATE simulations:
* Beam level
    * Beam Delivery Type
    * Beam Type
    * Beam Radiation Type
    * Beam Energy
    * Beam Fluence Type
    * Direction Gantry Rotation
    * Direction Collimator Rotation
    * Collimator Rotation
    * Isocenter Position
    * Applicator ID (electron beam)
    * Applicator Type (electron beam)
    * Source Surface Distance (electron beam)
* Control Point Index
    * Gantry Angle
    * X Jaws Poistions
    * Y Jaws Positions
    * MLC Positions
    * Dose Rate
    
    


In [ ]:
import pydicom as dcm

plan = dcm.read_file("data/patient_RP.dcm")
print(plan)

In [ ]:
for beam in plan.BeamSequence:
    print('Beam Name: ', beam.BeamName)
    print('Beam Delivery Type: ', beam.BeamType)
    print('Beam Radiation Type: ', beam.RadiationType)
    print('Beam Energy: ', beam.ControlPointSequence[0].NominalBeamEnergy)
    print('Beam Fluence Mode: ', beam.PrimaryFluenceModeSequence[0].FluenceMode)
    
    for control_index in beam.ControlPointSequence:
        print('Control Index: ', control_index.ControlPointIndex)
        print('     *Gantry Angle: ', control_index.GantryAngle)
        print('     *Dose Rate: ', control_index.ReferencedDoseReferenceSequence[0].CumulativeDoseReferenceCoefficient)
        print('     *MLC: ', control_index.BeamLimitingDevicePositionSequence[0].LeafJawPositions) # bug to fix control index 0 MLC vlaues seem wrong

## Exercise - build a dictionary with beam parameters
* create a dictionary with 'Name' and 'ID' as keys (and PatientName, PatientID as values)
* add a key 'Beam' as a dictionary
* loop over the plan.BeamSequence and search for TreatmentDeliveryType == 'TREATMENT'
* search for a LeafJawPositions in the beam.ControlPointSequence
* add the LeafJawPositions to your dictionary as a fonction of the BeamName

In [ ]:
# complete here ...